In [ ]:
%matplotlib notebook
import sympy

### Symbolic algebra

Here we'll demonstrate a bit about the computer algebra system [SymPy](https://www.sympy.org).
SymPy does symbolic math, much like Maple or Mathematica, entirely in Python.
The core data type in SymPy is a *symbol*, from which you can build more complicated expressions.

In [ ]:
x = sympy.symbols('x')

In [ ]:
f = sympy.exp(x) + sympy.sin(x)
f

Now that we've created a symbolic representation of the function $x \cdot \sin x$, we'll want some way to actually plug in values of $x$ and evaluate it.
The simplest way to do this is to use the method `subs` of the expression `f`, which will substitute in numeric values for `x`.

In [ ]:
f.subs(x, 1.0)

SymPy also has symbolic representations of transcendental numbers like $e$ and $\pi$.

In [ ]:
f.subs(x, sympy.pi)

If we used a numerical approximation of $\pi$, which we can get from the package numpy, we'll get an inexact answer.

In [ ]:
import numpy as np
f.subs(x, np.pi)

What this shows is that SymPy is smart enough to recognize identities like $\sin(k \cdot \pi) = 0$.

The `subs` method works perfectly fine, but if we need to evaluate this function repeatedly on a lot of numeric arguments it's quite slow.
So there's another function call `lambdify` that will take in a symbolic function and return a numeric one.
The lambdified version is much faster to call over and over again.

In [ ]:
F = sympy.lambdify(x, f)

In [ ]:
F(np.pi)

If you know you'll be using numpy for doing big array operations, you can tell sympy that you want a lambdified function that'll work with numpy too:

In [ ]:
F = sympy.lambdify(x, f, modules='numpy')
X = np.array([0., 1/6, 1/4, 1/3, 1/2, 2/3, 3/4, 1.]) * np.pi
F(X)

### Symbolic differentiation

One of the most useful features of these kinds of tools is that they can calculate symbolic derivatives.

In [ ]:
g = sympy.diff(f, x)
g

If we could only calculate derivatives numerically by the method of finite differences, we could get really bad results because of floating-point underflow.

In [ ]:
G = sympy.lambdify(x, g, modules='numpy')
G(0.)

In [ ]:
(F(1e-16) - F(0.)) / 1e-16

Observe how if we put in a symbolic argument we get a symbolic result, and if we put in a numeric argument we get a numeric result:

In [ ]:
g.subs(x, sympy.pi)

In [ ]:
g.subs(x, np.pi)

### Symbolic integration

Differentiation is a "nice" problem in that, if you have an symbolic expression for an elementary function, its derivative is elementary and there is an algorithm that will terminate in finite time to come up with a symbolic expression for its derivative.
Integration is not nice.
There are elementary functions with no elementary anti-derivative, for example $f(x) = e^{-x^2}$.
Nonetheless, SymPy and other symbolic algebra systems usually have a rich set of heuristics for calculating anti-derivatives.

In [ ]:
sympy.integrate(f, x)

I tried to stump the algorithm but they have so many special functions built in that it's kind of hard!
For example, the anti-derivative of $e^{-x^2}$ isn't elementary, but it's important enough that people decided they'd call it the *error function* (this comes from statistics).

In [ ]:
h = sympy.exp(-x**2)

In [ ]:
sympy.integrate(h, x)

If you want to actually evaluate the integral over specified bounds rather than get an indefinite integral, you can pass those integration bounds.
The variable `sympy.oo` stands for $\infty$.

In [ ]:
np.float64(sympy.integrate(h, (x, 0., sympy.oo)))

### Several variables

So far we've only looked at functions of a single variable $x$.
But we could just as easily use more symbols to look at functions of several variables.

In [ ]:
x, y = sympy.symbols('x y')

In [ ]:
f = x**2 - y**2
f

In [ ]:
sympy.diff(f, x)

In [ ]:
sympy.diff(f, y)

You can calculate higher-order derivatives and mixed derivatives too.

In [ ]:
sympy.diff(f, x, y)

In [ ]:
sympy.diff(f, x, x) + sympy.diff(f, y, y)

This is a *very* handy trick when you want to test whether your PDE solver is working right.
For example, let's say you wrote some code to solve the Poisson equation

$$-\nabla\cdot k\nabla u = f$$

where $k$ is a conductivity coefficient and $f$ is a right-hand side.
You could painstakingly pick some $k$ and $f$ so that this problem has an exact solution.
Or you could take any random old $k$, any random old $u$, and then *define* a right-hand side $f$ through the last equation.
You've already picked what the true honest $u$ will be, so you know exactly what to compare the result of your numerical solver with.
This is called the **method of manufactured solutions** and it's the standard way that the professionals make sure the PDE solvers do what they're supposed to.
For complex problems, doing all that algebra by hand is tedious and error-prone, but with a symbolic algebra system it's super easy.

### Finite element analysis

Filling matrices and vectors for the finite element method is really error-prone.
You can use a tool like SymPy to automate a lot of that.
For example, let's say we wanted to solve the boundary value problem

$$-u'' = f.$$

Eventually we're going to have to evaluate a bunch of integrals that look like

$$A_{ij} = \int_0^1 \phi_i'\cdot \phi_j'\, dx$$

where $\phi_i$ are the shape functions.
For linear shape functions this is easy to do by hand, but for quadratic or higher shape functions it starts to get awful.
But SymPy can save us from doing all that!

First, let's make a list of the quadratic basis functions on the interval.

In [ ]:
x = sympy.symbols('x')
shape_functions = [
    (1 - x) * (1 - 2 * x),
    4 * x * (1 - x),
    x * (2 * x - 1)
]

To make sure we tabulated them right, we can plot them.

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
X = np.linspace(0., 1., 51)
for index, function in enumerate(shape_functions):
    f = sympy.lambdify(x, function, modules='numpy')
    axes.plot(X, f(X), label=index)
axes.legend(loc='upper right')

Finally, we can evaluate all the entries of the element stiffness matrix by iterating over the shape functions twice!

In [ ]:
A = np.zeros((3, 3))
for i, ϕ_i in enumerate(shape_functions):
    for j, ϕ_j in enumerate(shape_functions):
        integrand = sympy.diff(ϕ_i) * sympy.diff(ϕ_j)
        A[i, j] = sympy.integrate(integrand, (x, 0., 1.))

In [ ]:
A

### Conclusion

SymPy and symbolic algebra systems generally are really nice tools for automating away tedious and error-prone math.
The example above shows how you might use symbolic algebra to automate one of the most important (and most error-prone) parts of finite element analysis: calculating local stiffness matrices.
The software package FEniCS was the first to take this idea to its logical conclusion: scientists like you should be able to specify symbolically what problem you want to solve, and the software will automatically fill matrices and vectors for you.
**Combining symbolic and numerical analysis is the biggest current trend in computational science today.**
In the following examples, I'll use Firedrake, an offshoot of FEniCS, to show how that works.